---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [67]:
import pandas as pd
import re
pd.set_option('display.max_rows', 500)
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
x=list(range(0,500))
df_obj=pd.DataFrame(df)
df_obj.columns=['note']
df_obj['note'].astype(str)
#df_obj.drop(df_obj.index[7]).reindex()
df_obj['note']=df_obj['note'].str.replace('10/24/89 CPT Code: 90801 - Psychiatric Diagnos...','10/24/89')
df_obj

,note
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...
5,.Per 7/06/79 Movement D/O note:\n
6,"4, 5/18/78 Patient's thoughts about current su..."
7,10/24/89Interview\n
8,3/7/86 SOS-10 Total Score:\n
9,(4/10/71)Score-1Audit C Score Current:\n


In [73]:
def date_sorter():
    df_obj['note'].astype(str)
    
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    
#    
#     df_obj['note']=df_obj['note'].str.findall(r'\d{1,2}[/]\d{1,2}[/]\d{2,4}')
#     

        #Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 

    # df_obj['note']= df_obj['note'].str.findall(r'(?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{1,2}(?:-|\.|\s|,)\s?\d{2,4}')


 #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    
    #df_obj['note']= df_obj['note'].str.findall(r'\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)[a-z]*(?:.|,)\s?\d{1,2}')
    
    
 # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    #df_obj['note']= df_obj['note'].str.findall(r'(?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)\s\d{2}[a-z]*(?:,)\s\d{4}')
    
    #Feb 2009; Sep 2009; Oct 2010
    #df_obj['note']= df_obj['note'].str.findall(r'(?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)\s\d{4}')
    
#     #6/2008; 12/2009
#     pattern1=df_obj['note'].str.extract(r'(\d{1,2}[/]\d{1,2}[/]\d{2,4})')
#     pattern2=df_obj['note'].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{1,2}(?:-|\.|\s|,)\s?\d{2,4})')
#     pattern3=df_obj['note'].str.extract(r'(\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)[a-z]*(?:.|,)\s?\d{1,2})')
#     pattern4=df_obj['note'].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)\s\d{2}[a-z]*(?:,)\s\d{4})')
#     pattern5=df_obj['note'].str.extract(r'((?:Jan|Feb|Mar|Apr|May|Ju|Jul|Au|Sep|Oct|Nov|Dec)\s\d{4})')
#     pattern6=df_obj['note'].str.extract(r'([1-2]\d{3})')
   
#     date=pd.to_datetime(pattern1.fillna(pattern2).fillna(pattern3).fillna(pattern4).fillna(pattern5).fillna(pattern5).fillna(pattern6).replace('Decemeber','December',regex=True).replace('Janaury','January',regex=True))
    
#     return pd.Series(date.sort_values().index)

    import re
    from calendar import month_name
    import dateutil.parser
    from datetime import datetime

    # There are 4 formats for the dates: 1) Dates are in numbers 2) Dates are in text 3) Dates without days, only month and year
    # 1) Dates are in numbers
    format_one = df.str.extract(r"((?:\d{1,2})(?:(?:\/|-)\d{1,2})(?:(?:\/|-)\d{2,4}))")
    # 2) Dates are in text
    format_two = df.str.extract(r"((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})")  
    # 3) Dates without days, only month and year
    format_three = df.str.extract(r'((?:\d{1,2}(?:-|\/))?\d{4})')
    dates = pd.to_datetime(format_one.fillna(format_two).fillna(format_three).replace('Decemeber','December',regex=True).replace('Janaury','January',regex=True))

    return pd.Series(dates.sort_values().index)
date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46      49
47     317
48      11
49     319
50      40
51     418
52     165
53     370
54     382
55       3
56      50
57     363
58     219
59     465
60     237
61      23
62     342
63     204
64     258
65     315
66      27
67      93
68      17
69     488
70     303
71     283
72     395
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     369
82     493
83     318
84     239
85     148
86     105
87     336
88       6
89     200
90      81